In [ ]:
%%capture
%pip install gurobipy
import warnings
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import Model
from gurobipy import GRB
warnings.filterwarnings('ignore')

In [ ]:
params = {
"WLSACCESSID": 'f9ac2e06-e48b-467c-b8e7-8348ea405464',
"WLSSECRET": '5fb89836-b582-4171-85ef-26b25b67790f',
"LICENSEID": 2610558,
}
env = gp.Env(params=params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2610558
Academic license 2610558 - for non-commercial use only - registered to sh___@utexas.edu


In [ ]:
# Question 2

from gurobipy import Model, GRB

# Defining the model
model = Model("Oil_Transport")

# Sets
sources = ["Well1", "Well2"]
intermediate = ["Mobile", "Galveston"]
destinations = ["NY", "LA"]

# Supply limits
supply = {"Well1": 150000, "Well2": 200000}

# Demand requirements
demand = {"NY": 140000, "LA": 160000}

# Shipping costs
costs = {
    ("Well1", "Mobile"): 10, ("Well1", "Galveston"): 13, ("Well1", "NY"): 30, ("Well1", "LA"): 28,
    ("Well2", "Mobile"): 15, ("Well2", "Galveston"): 12, ("Well2", "NY"): 26, ("Well2", "LA"): 25,
    ("Mobile", "NY"): 16, ("Mobile", "LA"): 17,
    ("Galveston", "NY"): 14, ("Galveston", "LA"): 13
}

# Decision variables
flow = model.addVars(costs.keys(), vtype=GRB.CONTINUOUS, name="flow")

# Supply constraints
for s in sources:
    model.addConstr(sum(flow[s, d] for d in intermediate + destinations if (s, d) in costs) <= supply[s], name=f"supply_{s}")

# Demand constraints
for d in destinations:
    model.addConstr(sum(flow[s, d] for s in sources + intermediate if (s, d) in costs) == demand[d], name=f"demand_{d}")

# Flow conservation at intermediate nodes
for i in intermediate:
    model.addConstr(sum(flow[s, i] for s in sources if (s, i) in costs) == sum(flow[i, d] for d in destinations if (i, d) in costs), name=f"balance_{i}")

# Objective function: minimize transportation cost
model.setObjective(sum(flow[i, j] * costs[i, j] for (i, j) in costs), GRB.MINIMIZE)

# Solving the model
model.optimize()

# Printing the results
if model.status == GRB.OPTIMAL:
    print("Optimal shipment strategy:")
    for (i, j) in costs:
        if flow[i, j].x > 0:
            print(f"Ship {flow[i, j].x:.2f} barrels from {i} to {j}")
    print(f"Minimal transport cost: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x0c0b4124
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 2e+05]
Presolve time: 0.01s
Presolved: 6 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0400000e+06   3.000000e+05   0.000000e+00      0s
       3    7.6400000e+06   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.640000000e+06
Optimal shipment strategy:
Ship 140000.00 barrels from Well1 to Mobile
Ship 160000.00 barrels fro

In [ ]:
# Question 3

# Defining the graph as (start_node, end_node): travel_time
edges = {
    (1, 2): 5,
    (2, 3): 3,
    (2, 4): 2,
    (3, 5): 4,
    (5, 6): 6,
    (5, 7): 8,
    (6, 7): 3
}

nodes = {1, 2, 3, 4, 5, 6, 7}
source = 1
destination = 7

# Create Gurobi model
model = Model("Shortest Path")

# Decision variables: x[i, j] = 1 if the edge (i, j) is used in the path, 0 otherwise
x = model.addVars(edges.keys(), vtype=GRB.BINARY, name="x")

# Objective: Minimize total travel time
model.setObjective(sum(edges[i, j] * x[i, j] for i, j in edges), GRB.MINIMIZE)

# Flow conservation constraints
for node in nodes:
    if node == source:
        model.addConstr(sum(x[i, j] for i, j in edges if i == node) == 1)  # Outflow from source
    elif node == destination:
        model.addConstr(sum(x[i, j] for i, j in edges if j == node) == 1)  # Inflow to destination
    else:
        model.addConstr(
            sum(x[i, j] for i, j in edges if i == node) - sum(x[i, j] for i, j in edges if j == node) == 0
        )  # Flow conservation at intermediate nodes

# Solving the model
model.optimize()

# Extracting and displaing results
if model.status == GRB.OPTIMAL:
    print("Shortest Path:")
    for i, j in edges:
        if x[i, j].x > 0.5:
            print(f"{i} -> {j}")
    print(f"Minimum travel time: {model.objVal}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 7 columns and 14 nonzeros
Model fingerprint: 0x91407717
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%
Shortest Path:
1 -> 2
2 -> 3
3 -> 5
5 -> 7
Minimum travel time: 20.0


In [ ]:
# Question 4
# I have done Q4 using 2 methods, they give the same answer!

# Create a new model
model = Model("Pipeline_Transportation")

# Define the network
# Nodes: 1 through 9
nodes = range(1, 10)

# Defining the directed graph with capacities
edges = {
    (1, 2): 50, (1, 5): 20,
    (2, 3): 20, (2, 6): 25,
    (3, 4): 25, (3, 7): 10,
    (3, 8): 15, (4, 9): 12,
    (6, 5): 15, (6, 7): 15,
    (7, 8): 15, (8, 9): 28
}

supply = {1: 100}  # Source node supply
demand = {5: 20, 9: 35}  # Minimum required demands

# Extract all unique nodes from edges
nodes = set(i for i, j in edges.keys()) | set(j for i, j in edges.keys())

# Create variables for flow through each edge
flow = model.addVars(edges.keys(), lb=0, name="flow")

# Set capacity constraints
for (i, j) in edges:
    flow[i, j].ub = edges[i, j]

# Source node constraint
model.addConstr(
    sum(flow[i, j] for i, j in edges.keys() if i == 1) <= supply[1],
    name="Source_Constraint"
)

# Flow conservation constraints
for node in nodes - {1, 5, 9}:  # Intermediate nodes
    model.addConstr(
        sum(flow[i, j] for i, j in edges.keys() if j == node) ==
        sum(flow[i, j] for i, j in edges.keys() if i == node),
        name=f"Flow_Conservation_{node}"
    )

# Demand constraints at destination nodes
for d in demand:
    model.addConstr(
        sum(flow[i, j] for i, j in edges.keys() if j == d) >= demand[d],
        name=f"Demand_Node_{d}"
    )

# Objective: Maximize total flow reaching demand nodes
model.setObjective(
    sum(flow[i, j] for i, j in edges.keys() if j in demand),
    GRB.MAXIMIZE
)

# Optimizing
model.optimize()

# Printing results
if model.status == GRB.OPTIMAL:
    print("Optimal Flow Plan:")
    for (i, j) in edges.keys():
        if flow[i, j].x > 0.001:  # Only print non-zero flows
            print(f"Flow from {i} to {j}: {flow[i, j].x:.2f}")

    print(f"\nMaximum transported fluid: {model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xbcf3f920
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 5e+01]
  RHS range        [2e+01, 1e+02]
Presolve removed 4 rows and 5 columns
Presolve time: 0.01s
Presolved: 5 rows, 7 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+01   1.650000e+01   0.000000e+00      0s
       6    6.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.500000000e+01
Optimal Flow Plan:
Flow from 1 to 2: 45.00
Flow from 1 to 5: 20.00
Flow from 2 to 3: 20.00
Flow from 2 to 6: 25.00
Flow from 3 to 

In [ ]:
# Question 4

# Create a new model
model = gp.Model("Pipeline_Transportation")

# Define the network
# Nodes: 1 through 9
nodes = range(1, 10)

# Edges (pipelines) with their capacities based on the image
edges = [
    (1, 2, 50),  # Node 1 to 2 with capacity 50
    (1, 5, 20),  # Node 1 to 5 with capacity 20
    (2, 3, 20),  # Node 2 to 3 with capacity 20
    (2, 6, 25),  # Node 2 to 6 with capacity 25
    (3, 4, 25),  # Node 3 to 4 with capacity 25
    (3, 7, 10),  # Node 3 to 7 with capacity 10
    (3, 8, 15),  # Node 3 to 8 with capacity 15
    (4, 9, 12),  # Node 4 to 9 with capacity 12
    (6, 5, 15),  # Node 6 to 5 with capacity 15
    (6, 7, 15),  # Node 6 to 7 with capacity 15
    (7, 8, 15),  # Node 7 to 8 with capacity 15
    (8, 9, 28)   # Node 8 to 9 with capacity 28
]

# Creating flow variables
flows = {}
for (u, v, capacity) in edges:
    flows[u, v] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS,
                              name=f'flow_{u}_{v}')

# Sourcing constraint (Node 1)
model.addConstr(
    gp.quicksum(flows[1, j] for (i, j, _) in edges if i == 1) <= 100,
    "Source_Capacity"
)

# Flow conservation for intermediate nodes (2, 3, 4, 6, 7, 8)
for node in [2, 3, 4, 6, 7, 8]:
    incoming = gp.quicksum(flows[i, node] for (i, j, _) in edges if j == node)
    outgoing = gp.quicksum(flows[node, j] for (i, j, _) in edges if i == node)
    model.addConstr(incoming == outgoing, f"Flow_Conservation_{node}")

# Demand constraints for nodes 5 and 9
model.addConstr(
    gp.quicksum(flows[i, 5] for (i, j, _) in edges if j == 5) >= 20,
    "Demand_Node_5"
)
model.addConstr(
    gp.quicksum(flows[i, 9] for (i, j, _) in edges if j == 9) >= 35,
    "Demand_Node_9"
)

# Objective: Maximize total flow to demand nodes
model.setObjective(
    gp.quicksum(flows[i, 5] for (i, j, _) in edges if j == 5) +
    gp.quicksum(flows[i, 9] for (i, j, _) in edges if j == 9),
    GRB.MAXIMIZE
)

# Solving the model
model.optimize()

# Printing results
print("Optimization Status:", model.status)
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution:")
    print("Maximum Total Flow:", model.objVal)

    print("\nFlow Distribution:")
    for (u, v, _) in edges:
        if flows[u, v].x > 0.001:  # Only print non-zero flows
            print(f"Flow {u}->{v}: {flows[u, v].x:.2f}")

    print("\nTotal Flow to Destinations:")
    print(f"Node 5: {sum(flows[i, 5].x for (i, j, _) in edges if j == 5):.2f}")
    print(f"Node 9: {sum(flows[i, 9].x for (i, j, _) in edges if j == 9):.2f}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xbcf3f920
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 5e+01]
  RHS range        [2e+01, 1e+02]
Presolve removed 4 rows and 5 columns
Presolve time: 0.01s
Presolved: 5 rows, 7 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+01   1.650000e+01   0.000000e+00      0s
       6    6.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.500000000e+01
Optimization Status: 2

Optimal Solution:
Maximum Total Flow: 65.0

Flow Distribution:
Flow 1->2: 45.00
Flow 1->5: 20.00
Flow 2->3